# Warp a disk

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import numpy as np
from tqdm.auto import tqdm

import diskwarp
from diskwarp import helper

%matplotlib widget
%config InlineBackend.figure_format = 'retina'

## General Definitions of disk surface and warp parameters

Calculate the interfaces and cell centers of the disk surface

In [ ]:
r_i = np.linspace(0.1, 150, 201)
r_c = 0.5 * (r_i[1:] + r_i[:-1])

In [ ]:
surf     = helper.get_surface(r_i, nphi=100)
p0_i     = surf['points_i']
p0_c     = surf['points_c']
ri       = surf['ri']
rc       = surf['rc']
phic     = surf['phic']
phii     = surf['phii']
nr, nphi = p0_c.shape[:-1]

Define the warp (inclination for each ring): logistic function from $i=0$ outside transitioning to `warp_max` at radius `r0` over a transition width of `dr`.

In [ ]:
warp_c = helper.warp(r_c, i_in=45, r0=75., dr=10.0)
warp_i = helper.warp(r_i, i_in=45, r0=75., dr=10.0)

Define each radius' twist

In [ ]:
twist_i = helper.twist(r_i, phi=0.0) + np.deg2rad(0.0)
twist_c = helper.twist(r_c, phi=0.0) + np.deg2rad(0.0)

Calculate the velocities of all centers and edges: we assume perfect keplerian rotation, so
$v_\phi \propto r^{-1/2} \propto \left(x^2 + y^2\right)^{-1/4}$

For testing, we might take the radial gradient out and just set $v_\phi = 1$.

Here, `v0_c` and `v0_i` contain the x-, y-, and z-component of the velocity.

In [ ]:
v0_c = (p0_c[:, :, 0]**2 + p0_c[:, :, 1]**2)**-0.25  # Keplerian
v0_c = 0.0 * v0_c + 1.0  ##### make it uniform ######
v0_c = v0_c[None, :, :] * np.array([-np.sin(phic), np.cos(phic), np.zeros_like(phic)])
v0_c = np.moveaxis(v0_c, 0, 2)

v0_i = (p0_i[:, :, 0]**2 + p0_i[:, :, 1]**2)**-0.25  # Keplerian
v0_i = 0.0 * v0_i + 1.0  ##### make it uniform ######
v0_i = v0_i[None, :, :] * np.array([-np.sin(phii), np.cos(phii), np.zeros_like(phii)])
v0_i = np.moveaxis(v0_i, 0, 2)

define the inclination of the observer as rotation around x

In [ ]:
inc_obs = np.deg2rad(30.0)
PA_obs  = np.deg2rad(0.0)
azi     = np.deg2rad(0.0)

## Apply the warp/twist

In [ ]:
p1_c = diskwarp.fmodule.apply_matrix2d(p0_c, warp_c, twist_c, inc_obs, PA_obs, azi)
v1_c = diskwarp.fmodule.apply_matrix2d(v0_c, warp_c, twist_c, inc_obs, PA_obs, azi)
p1_i = diskwarp.fmodule.apply_matrix2d(p0_i, warp_i, twist_i, inc_obs, PA_obs, azi)
v1_i = diskwarp.fmodule.apply_matrix2d(v0_i, warp_i, twist_i, inc_obs, PA_obs, azi)

In [ ]:
ir = np.array([50, 75, 100, 125, 150])
s_phi = 4

scale = .2 * r_c[ir] / (v1_c[ir, :, :]**2).sum(-1).mean(-1)
arr_1 = np.array([p1_c[ir, ::s_phi, :], p1_c[ir, ::s_phi, :] + scale[:, None, None] * v1_c[ir, ::s_phi, :]])

In [ ]:
X, Y, Z = p1_i.T

#C = Normalize()(ri[:, :].T) ## color by radius
C = Normalize()(v1_i[:, :, -1].T) ## color by vz

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(projection='3d')
ax.view_init(90, -90)
ax.set_proj_type('ortho')
surf = ax.plot_surface(X, Y, Z, facecolors=plt.cm.RdBu(C), shade=False, linewidth=0, alpha=0.6, rstride=1, cstride=1)

for _ir in range(arr_1.shape[1]):
    for iphi in range(arr_1.shape[2]):
        ax.plot3D(*arr_1[:, _ir, iphi, :].T, 'r', lw=1)

ax.set_xlim(-r_i[-1] / 1.4, r_i[-1] / 1.4)
ax.set_ylim(-r_i[-1] / 1.4, r_i[-1] / 1.4)
ax.set_zlim(-r_i[-1] / 1.4, r_i[-1] / 1.4)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_axis_off()

Make a video

In [ ]:
from pathlib import Path
import subprocess

fpath = Path('frames')
fpath.mkdir(exist_ok=True)

for i in tqdm(np.arange(0, 360)):
    ax.view_init(20.33, i)
    fig.savefig(fpath / f'frame_{i:03d}.png')

p = subprocess.getoutput('ffmpeg -y -framerate 30 -i frames/frame_%03d.png -c:v libx264 -crf 23 -pix_fmt yuv420p video.mp4')

In [ ]:
from IPython.display import Video
Video('video.mp4', width=500, height=500, html_attributes='loop controls autoplay') 

## Now interpolate on sky plane

In [ ]:
_gx = np.linspace(-r_i[-1], r_i[-1], 200)
_gy = np.linspace(-r_i[-1], r_i[-1], 210)
img_xi, img_yi = np.meshgrid(_gx, _gy, indexing='ij')
img_xc = 0.5 * (img_xi[1:, 1:] + img_xi[:-1, 1:])
img_yc = 0.5 * (img_yi[1:, 1:] + img_yi[1:, :-1])

In [ ]:
X, Y, Z = p1_i.T
vxi, vyi, vzi = v1_i.T
img_z, img_v = diskwarp.fmodule.interpolate_grid(X, Y, Z, vzi, img_xc, img_yc)
_,     img_r = diskwarp.fmodule.interpolate_grid(X, Y, Z, ri.T, img_xc, img_yc)

Get rid of the values outside of the domain

In [ ]:
img_v[img_v==img_v[0,0]] = np.nan
img_r[img_r==img_r[0,0]] = np.nan

In [ ]:
np.sin(np.deg2rad(30+45))

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(10,4), gridspec_kw={'width_ratios': [20, 20, 1]})
axs[1].sharex(axs[0])
axs[1].sharey(axs[0])

c1 = axs[0].contour(img_xc, img_yc, img_r, 20, colors='k', linewidths=.5)
c2 = axs[0].contourf(img_xc, img_yc, img_r, 50, cmap='RdBu')
surf = axs[1].pcolormesh(img_xi, img_yi, img_v, cmap='RdBu', vmax=-1, vmin=1)

fig.colorbar(surf, cax=axs[2])

axs[0].set_title('radius')
axs[1].set_title('radial velocity')

for ax in axs[:2]:
    ax.set_xlim(-150, 150)
    ax.set_ylim(-150, 150)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_aspect(1)

In [ ]:
fig.savefig('warp.pdf', transparent=True, bbox_inches='tight')